In [86]:
import numpy as np
import scipy.special as sp


In [159]:
def CarToSph(xyz):
    if len(xyz.shape)==2:
        x, y, z = xyz[:,0], xyz[:,1], xyz[:,2]
    elif len(xyz.shape)==1:
        x, y, z = xyz[0], xyz[1], xyz[2]
    r = np.sqrt(x**2+y**2+z**2)
    theta = np.arccos(z/r)
    phi = np.arctan2(y, x)
    return r, theta, phi


def SphToCar(r, theta, phi):
    x = r*np.cos(phi)*np.sin(theta)
    y = r*np.sin(phi)*np.sin(theta)
    z = r*np.sin(theta)
    return x, y, z


In [3]:
N = 1000
particle = np.random.uniform(size=(N, 4))


In [96]:
eps = 10**-2
p = int(np.ceil(-np.log(eps)/np.log(3**0.5)))


In [72]:
level = int(np.ceil(np.log2(N)/3)) + 1
idx_particle = -np.ones(shape=(N, level, 3), dtype=np.int32)
for i in range(N):
    idx_particle[i, 0, :] = 0
    residu = particle[i, :3]
    for j in range(1, level):
        split = np.int32(residu > 0.5)
        idx_particle[i, j, :] = idx_particle[i, j-1, :]*2+split
        residu = 2*residu-split


In [102]:
def XYZToL(idx, level):
    n = 2**level
    return idx[2]*n*n+idx[1]*n+idx[0]


def LToXYZ(l, level):
    n = 2**level
    z = l//(n*n)
    y = (l-z*n*n)//n
    x = (l-z*n*n) % n
    return np.array([x, y, z])


In [103]:
LToXYZ(XYZToL([541, 342, 247], 10), 10)


array([541, 342, 247])

In [73]:
tree_idx = [[[] for j in range(8**i)] for i in range(level)]
for i in range(level):
    for j in range(N):
        tree_idx[i][XYZToL(idx_particle[j, i, :], i)].append(j)


In [81]:
def CellCenter(idx, level):
    return idx*2**(-level)+2**(-level-1)


In [82]:
def NeighboursRange(center_idx, center_level):
    return [np.maximum(0, center_idx[center_level, :]-1), np.minimum(2**center_level-1, center_idx[center_level, :]+1)]


def NeighboursChildRange(center_idx, center_level):
    return [np.maximum(0, 2*(center_idx[center_level, :]-1)), np.minimum(2**(center_level+1)-1, 2*center_idx[center_level, :]+3)]


In [78]:
idx_particle[0, 2, :], idx_particle[0, 3, :]


(array([2, 3, 2]), array([5, 7, 4]))

In [83]:
NeighboursRange(idx_particle[0, :, :], 2)


[array([1, 2, 1]), array([3, 3, 3])]

In [84]:
NeighboursChildRange(idx_particle[0, :, :], 2)


[array([2, 4, 2]), array([7, 7, 7])]

In [85]:
NeighboursRange(idx_particle[0, :, :], 3)


[array([4, 6, 3]), array([6, 7, 5])]

In [84]:
def InteractionList(cell_idx, N_level):
    for i in range(N_level):
        # Range of parent's neighbours
        incluide = NeighboursChildRange(cell_idx, i-1)
        exclude = NeighboursRange(cell_idx, i)  # Range of neighbours


In [176]:
def Y(m, n, theta, phi):
    y = np.sqrt(sp.factorial(n-abs(m))/sp.factorial(n+abs(m)))\
        * sp.lpmn(abs(m), n, np.cos(theta))[0][-1][n]*np.cos(m*phi)
    return y


def M(m, n, mass, relat_xyz):
    corr_sph = CarToSph(relat_xyz)
    sum = 0
    for k in range(mass.shape[0]):
        sum += mass[k]*corr_sph[0][k]**n * \
            Y(-m, n, corr_sph[1][k], corr_sph[2][k])
    return sum


def phi(relat_xyz, M_coeff):
    corr_sph = CarToSph(relat_xyz)
    tot = 0
    for n in range(M_coeff.shape[0]):
        for m in range(0, n+1):  # range(-n, n+1):
            if m == 0:
                tot += M_coeff[n, m]*Y(m, n, corr_sph[1],
                                       corr_sph[2])/(corr_sph[0]**(n+1))
            else:
                tot += 2*M_coeff[n, m]*Y(m, n, corr_sph[1],
                                         corr_sph[2])/(corr_sph[0]**(n+1))
    return tot


In [134]:
# O(NlogN) scheme

M_tree = [np.zeros(shape=(8**i, p+1, p+1)) for i in range(level)]
for i in range(level):
    for j in range(8**i):
        idx = np.array(tree_idx[i][j])
        if len(idx) == 0:
            continue
        XYZ = particle[idx, :3] - CellCenter(LToXYZ(j, i), i)
        mass = particle[idx, 3]
        for n in range(p+1):
            for m in range(0, n+1):
                M_tree[i][j, n, m] = M(m, n, mass, XYZ)

In [180]:
potential = np.zeros(N)
for i in range(N):
    incl = NeighboursRange(idx_particle[i, :, :], level-1) # Direct evaluate
    for x in range(incl[0][0], incl[1][0]+1):
        for y in range(incl[0][1], incl[1][1]+1):
            for z in range(incl[0][2], incl[1][2]+1):
                index = np.array(tree_idx[level-1][XYZToL([x, y, z], level-1)])
                if len(idx) == 0:
                    continue
                for id in index:
                    if id == i:
                        continue
                    dXYZ = particle[id, :3] - particle[i, :3]
                    r = np.sqrt(np.sum(dXYZ**2))
                    potential[i] += particle[id, 3]/r
    for j in range(2, level):
        # Range of parent's neighbours
        incl = NeighboursChildRange(idx_particle[i, :, :], j-1)
        excl = NeighboursRange(idx_particle[i, :, :], j)  # Range of neighbours
        for x in range(incl[0][0], incl[1][0]+1):
            for y in range(incl[0][1], incl[1][1]+1):
                for z in range(incl[0][2], incl[1][2]+1):
                    if excl[0][0] <= x and excl[1][0] >= x and excl[0][1] <= y and excl[1][1] >= y and excl[0][2] <= z and excl[1][2] >= z:
                        continue
                    RelativeXYZ = particle[i, :3] - CellCenter(np.array([x, y, z]), i)
                    potential[i] += phi(RelativeXYZ, M_tree[j][XYZToL([x, y, z], j), :, :])
                    

In [171]:
potential_direct = np.zeros(N)
for i in range(N):
    for j in range(N):
        if i == j:
            continue
        dXYZ = particle[j, :3] - particle[i, :3]
        r = np.sqrt(np.sum(dXYZ**2))
        potential_direct[i] += particle[j, 3]/r


In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10,8))
plt.scatter(potential,potential_direct,s=2)
plt.axis("equal")
plt.xlim([200,1800])
plt.xlabel("FMM")
plt.ylabel("Direct-N")